# Understanding Dropout

## 학습 목표

Drop-out이란? 무작위로 신경망의 유닛을 삭제시키는 기법

> 왜 정규화로 잘 작동하는지 알아보기

## Why does drop-out work?

### Intuition1
* drop-out은 신경망의 유닛을 무작위로 삭제하는 것, 모든 반복마다 더 작은 신경망에서 작업하게된다.<br>
더 작은 신경망을 사용하는 것이 정규화의 효과를 주는 것처럼 보인다.

<center><img src="images/7_single_unit.PNG"height="300px" width="300px" ></center>
<p align="center"> Figure1. single unit </p>

### Intuition2 : single unit 관점
- unit이 해야하는 일 : 입력을 받아 의미있는 출력을 생성하는 것 <br>

- Drop-out을 통해 입력은 무작위로 삭제될 수 있는데 어떤 경우는 이 두 Unit이 삭제되고 어떤 경우에는 다른 Unit이 삭제된다. <br>

- 보라색 unit : 어떤 feature에도 의존할 수 없다.  <br>
그 feature이 무작위로 바뀌거나 feature의 고유한 입력이 무작위로 바뀔 수 있기때문에 <br>
특정 입력에 모든 것을 걸 수 없는, 특정 입력에 유난히 큰 가중치를 부여하기가 꺼려지는 상황 <br>
> 따라서 이 네 개의 입력 각각에 가중치를 분산시키는 편이 낫고 가중치를 분산시킴으로써 가중치의 노름의 제곱값이 줄어들게 된다.

- L2 정규화에서 봤던 것처럼 드롭아웃을 구현하는 효과는 가중치를 줄이는 것이고 <br>
L2 정규화처럼 overfitting을 막는데 도움이 된다.

> drop-out은 예전에 L2 정규화의 adaptive form으로 보여지기도 했다.<br>
그러나 L2 정규화에서 다른 가중치는 다르게 취급한다. 그 가중치에 곱해지는 활성화의 크기에 따라 다르다.

#### 정리

- drop-out은 L2 정규화와 비슷한 효과를 보여줄 수 있다.

- L2 정규화가 다른 가중치에 적용된다는 것과 서로 다른 크기의 입력에 더 잘 적응한다는것만 다른 부분이다.

### Intuition3

<center><img src="images/7_NN.PNG"height="500px" width="500px" /></center>
<p align="center"> Figure2. Neural Networks </p>

- keep_prob : 각 층에 해당 unit을 유지할 확률
> 층마다 keep_prob을 바꾸는 것도 가능

* overfitting의 우려가 많은 층 (parameter가 많은 층)
> overfitting을 줄이기위해 $W^{[2]}$는 상대적으로 낮은 keep_prob를 설정한다 &rarr; 0.5


* overfitting의 우려가 적은 층 
> 더 높은 keep_prob을 설정해도 된다 &rarr; 0.7


* overfitting의 우려가 없는 층 (입력층, 출력층)
> keep_prob = 1로 설정해도 된다
> 모든 unit을 유지하고 해당 층에서는 드롭아웃을 사용하지 않는다는 의미 <br>
>> 입력층은 feature의 절반 이상을 삭제하고싶지 않기때문에 drop-out을 자주 사용하지 않는것이 좋다. <br>
> 입력층에 대해 keep_prob = 1이 가장 흔한값이다. (0.9를 사용하기도 한다.)

**keep_prob을 높게 설정하는 것은 L2 정규화에서 더 많은 정규화가 필요한 층에서 parameter λ를 증가시키는것과 비슷하다.**


#### 정리
* 다른 층보다 과대적합의 우려가 더 큰 층에 대해서는 다른 층보다 더 낮은 값의 keep_prob을 설정할 수 있다.
* 단점 : 교차 검증을 위해 더 많은 hyperparameter가 생긴다.
* 대안 : 어떤 층에는 드롭아웃을 적용하고 어떤 층에는 적용하지 않아서 매개변수를 드롭아웃을 적용한 층에 대한 keep_prob 하나만 갖는 것

## Implemetational tips

### Computer Vision
- 컴퓨터 비전은 아주 많은 픽셀 값을 모두 사용하기 때문에 대부분의 경우 데이터가 부족하다.<br>
따라서 컴퓨터 비전에서 drop-out이 매우 빈번하게 사용된다.<br>
최근 비전 분야의 연구원들은 거의 항상 기본값으로 drop-out을 사용한다.<br>

- 그러나 여기서 기억해야 할 것은 drop-out은 정규화 기법이고 overfitting을 막는데 도움을 준다.<br>
따라서 네트워크가 과대적합의 문제가 생기기 전까지는 드롭아웃을 사용하지 않을 것이다.<br>

> **컴퓨터 비전은 충분한 데이터가 없기 때문에 거의 대부분 overfitting이 일어나고 drop-out을 많이 사용하는 이유이다.**

## Drop-out 단점
- 비용함수 J가 더 이상 잘 정의되지 않는다. 
> 모든 반복마다 무작위로 한 뭉치의 노드들을 삭제하기 때문에 경사 하강법의 성능을 이중으로 확인한다면 모든 반복에서 잘 정의된 비용함수 J가 하강하는지 확인하는게 어려워진다.

&rarr; 최적화하는 비용함수는 잘 정의되지 않아 계산하기 어렵다.

따라서 다음의 모양인지 확인해서 디버깅하는게 어려워진다.

<center><img src="images/7_cost.PNG"height="300px" width="300px" /></center>


<center> Figure3. cost function J </center>

- 보통 keep_prob을 1로 설정해서 드롭아웃 효과를 멈추고 코드를 실행시켜 J가 단조감소하는지 확인한다.
- 그리고 drop-out 효과를 다시 주고 drop-out이 있을 때 코드를 바꾸지 않도록 한다. 
> 왜냐하면 drop-out이 있을 때 코드와 경사 하강법이 잘 작동하는지 함수만 보고 확인하는 것 외에 다른 방법이 필요하기 때문이다.

---
# 참고
[Andrew Ng-Understanding Dropout](https://www.youtube.com/watch?v=ARq74QuavAo&list=PLkDaE6sCZn6Hn0vK8co82zjQtt3T2Nkqc&index=7)
